In [67]:
# plots
%matplotlib inline

import matplotlib.pyplot as plt
from pylab import rcParams
import math
rcParams['figure.figsize'] = 15, 8

import math
import scipy

import json

# Pandas
import pandas as pd
from pandas import DataFrame

# math api
import numpy as np

# time lib
import time
import datetime as dt

NOW = dt.datetime.now()


import seaborn as sns
sns.set()




In [39]:
flat_data = pd.read_csv("../ressources/data/financial_sells_1000000.csv")
flat_data = flat_data.fillna(0).replace('-', np.NaN)
flat_data = flat_data.drop(columns=['Unnamed: 0'])
flat_data.head()

,company_name,company_id,country,sector_name,product_name,NBI,date
0,Jackson PLC,1,Espagne,Electrical industry,Export Finance,51.550912,2016-04-14
1,"Leonard, Rosario and Rollins",2,United Kinkdom,Chemical Industry,Fund Distribution Services,80.574456,2015-08-23
2,Hamilton-Howard,3,United State,Energy industry,Asset Management,56.375726,2018-07-27
3,"Turner, Richardson and Gomez",4,Italy,Metallurgical industry,Equity,54.201823,2016-11-29
4,Parsons Ltd,5,Corse,Automotive,Clearing Services,16.495136,2018-06-23


# first implementation of Cross-Selling

In [76]:
dict_combinaison = dict()

for sector in flat_data['sector_name'].unique(): 
    for country in flat_data['country'].unique():
        
        
        query_fetch = 'country == "{}" and sector_name == "{}"'.format(country, sector)
        result_query = flat_data.query(query_fetch)
        all_products = result_query['product_name'].unique()
        
        for company_name in result_query['company_name'].unique():
            bought_products = set(flat_data.query('company_name == "{}"'.format(company_name))['product_name'].unique())
            all_products = set(all_products)
            unbought_products = all_products.symmetric_difference(bought_products)
            
            if( len(unbought_products) > 0 ):
                dict_combinaison[ company_name ] = {
                    'company_name': company_name,
                    'sector': sector,
                    'country': country,
                    'bought_product': bought_products,
                    'unbought_product': unbought_products,
                    'diff': len(unbought_products)
                }
# 07_cross_selling_output_bihavior
iterable = iter(dict_combinaison.values())
next(iterable)
        

{'company_name': 'Monroe, James and Peterson',
 'sector': 'Electrical industry',
 'country': 'Italy',
 'bought_product': {'Asset Based Financing', 'Hybrids', 'MCIB Financing'},
 'unbought_product': {'Export Finance'},
 'diff': 1}

# second implementation of cross_selling

In [11]:
def apply_cross_selling(_df):
    # count all unique clients
    count_distinct_customers = _df['company_name'].nunique()

    result_cross_selling = dict()

    # for each item, compare with otel items
    for product_1 in _df['product_name'].unique(): 
        
        #print( product_1 )
        
        #count all users who bougth this item1
        query_product_1 = "product_name == '{0}'".format(product_1)
        result_query_1 = _df.query(query_product_1)


        for product_2 in _df['product_name'].unique():
            dict_key1 = (product_2, product_1)
            dict_key2 = (product_1, product_2)
            score = 0

            if ( dict_key1 in result_cross_selling ):
                #print('already exists', end=' - ')
                continue

            #count all users who bougth this item1
            query_product_2 = "product_name == '{0}'".format(product_2)
            result_query_2 = _df.query(query_product_2)

            list_customer_product_1 = set( result_query_1['company_name'].unique() )
            list_customer_product_2 = set( result_query_2['company_name'].unique() )
            common_customers = list_customer_product_1.intersection(list_customer_product_2)

            score = len( common_customers ) / count_distinct_customers

            

            result_cross_selling[ dict_key1 ] = score
            result_cross_selling[ dict_key2 ] = score
            

apply_cross_selling(flat_data)

Liquidity Management
Credit
Loans syndication
Equity
Custody Services
Export Finance
Asset Servicing
ALD Car Renting and Fleet Management
Mergers and Acquisition
International Retail Banking
Commodity Financing
Asset Based Financing
Asset Management
Fund Distribution Services
Prime & Clearing
Life and Dammage insurance
Clearing Services
Commodities
Leverage Finance
Fund Administration
MCIB Financing
Issuer Services
General Purpose Financing
Hybrids
Others
Financing
Forex
